
# Data Cleaning


# Objectives 


- Improve data quality
- Handle missing data 


## Inputs

- outputs/datasets/datacollection/HousePrices.csv


## Outputs

- Train Set
- Test Set


## CRISP-DM 


"Data Preparation"

In [ ]:
import os
current_dir = os.getcwd()
current_dir

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

In [ ]:
current_dir = os.getcwd()
current_dir

In [ ]:
# Load data 

import pandas as pd
df = pd.read_csv("outputs/datasets/datacollection/HousePrices.csv")
df.head(10)

In [ ]:
# Variables with missing data. 

vars_with_missing_data = df.columns[df.isna().sum() > 0].to_list()
vars_with_missing_data

In [ ]:
df.loc[:, vars_with_missing_data].info()

In [ ]:
# Code can be found in the Churnometer Data Cleaning workbook.
from pandas_profiling import ProfileReport
if vars_with_missing_data:
    profile = ProfileReport(df=df[vars_with_missing_data], minimal=True)
    profile.to_notebook_iframe()
else:
    print("There are no variables with missing data")

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ppscore as pps 

# Use inline plotting for Jupyter
%matplotlib inline

def plot_heatmap(matrix, threshold, figsize=(20, 12), font_size=8, cmap="viridis", mask_upper=True):
    if matrix.shape[1] > 1:
        mask = np.zeros_like(matrix, dtype=bool)

        if mask_upper:
            mask[np.triu_indices_from(mask)] = True

        mask[abs(matrix) < threshold] = True

        plt.figure(figsize=figsize)
        sns.heatmap(matrix, annot=True, mask=mask, cmap=cmap, annot_kws={"size": font_size},
                    linewidths=0.5, linecolor="gray")
        plt.xticks(rotation=90)
        plt.yticks(rotation=0)
        plt.ylim(len(matrix.columns), 0)
        plt.show()

def compute_correlation_and_pps(df):
    spearman_corr = df.corr(method="spearman")
    pearson_corr = df.corr(method="pearson")

    pps_matrix_raw = pps.matrix(df)
    pps_matrix = pps_matrix_raw.pivot(index="y", columns="x", values="ppscore")

    pps_stats = pps_matrix_raw.query("ppscore < 1 ")["ppscore"].describe()
    print("PPS Summary Stats (for threshold selection):\n", pps_stats.round(3))

    return pearson_corr, spearman_corr, pps_matrix

def display_heatmaps(pearson_corr, spearman_corr, pps_matrix, corr_threshold, pps_threshold, figsize=(20,12), font_size=8):
    print("Spearman Correlation Heatmap (Montonic relationships):")
    plot_heatmap(pearson_corr, corr_threshold, figsize, font_size, cmap="magma")

    print("Pearson Correlation Heatmap (Linear relationships):")
    plot_heatmap(pearson_corr, corr_threshold, figsize, font_size, cmap="coolwarm")

    print("Power Predictive Score (PPS) Heatmap:")
    plot_heatmap(pps_matrix, pps_threshold, figsize, font_size, cmap="rocket_r", mask_upper=False)


In [ ]:
df_corr_pearson, df_corr_spearman, pps_matrix = compute_correlation_and_pps(df)

display_heatmaps(
    pearson_corr=df_corr_pearson, 
    spearman_corr=df_corr_spearman, 
    pps_matrix=pps_matrix, 
    corr_threshold=0.4, 
    pps_threshold=0.2, 
    figsize=(12, 10), 
    font_size=10
)


## Data Cleaning

### Function to create a DataFrame to hold missing data statistics. 

In [16]:
def evaluate_missing_data(df):
    """
    Analyse variables with missing values
    """
    missing_counts = df.isnull().sum()
    missing_percentage = (missing_counts / df.shape[0] * 100).round(2)
    

    # Create a DataFrame to hold the missing data statistics. 
    missing_data_df = pd.DataFrame({
        "RowsWithMissingData": missing_counts,
        "PercentageOfDataset": missing_percentage,
        "DataType": df.dtypes
    })

    # Filter to include only columns with missing data and sort.
    missing_data_df = missing_data_df[missing_data_df["PercentageOfDataset"] > 0]
    missing_data_df = missing_data_df.sort_values(by="PercentageOfDataset", ascending=False)

    return missing_data_df

In [ ]:
evaluate_missing_data(df)

- "EnclosedPorch" and "WoodDeckSF" have the most missing values at well over 80% each. I would deem these variables almost obsolete as they don't offer much in the way of predcting house prices. 

In [18]:
import pandas as pd

def plot_data_cleaning_effect(df_original, df_cleaned, variables_applied):
    """
    Visualizes the effect of data cleaning on selected variables by comparing distributions 
    between the original and cleaned datasets.

    Parameters:
        df_original (pd.DataFrame): The original dataset.
        df_cleaned (pd.DataFrame): The cleaned dataset.
        variables_applied (list): The variables to plot.
    """

    sns.set(style="whitegrid")
    
    # Identify categorical variables in the original dataset
    categorical_vars = df_original.select_dtypes(exclude=['number']).columns
    
    print("\n=====================================================================================")
    print(f"* Distribution Effect Analysis After Data Cleaning on the following variables:")
    print(f"{variables_applied} \n\n")
    
    for idx, var in enumerate(variables_applied, start=1):
        plt.figure(figsize=(15, 5) if var in categorical_vars else (10, 5))
        
        # Plot categorical variables as bar plots
        if var in categorical_vars:
            df1 = pd.DataFrame({"Type": "Original", "Value": df_original[var]})
            df2 = pd.DataFrame({"Type": "Cleaned", "Value": df_cleaned[var]})
            combined_df = pd.concat([df1, df2])
            
            sns.countplot(data=combined_df, x="Value", hue="Type", palette=["#01BAEF", "#B80C09"])
            plt.xticks(rotation=90)
        
        # Plot numerical variables as histograms
        else:
            sns.histplot(df_original[var], color="#432371", label="Original", kde=True, element="step")
            sns.histplot(df_cleaned[var], color="#FAAE7B", label="Cleaned", kde=True, element="step")
        
        plt.title(f"Distribution Plot {idx}: {var}")
        plt.legend()
        plt.show()

## CategoricalVariableImputer


- **Purpose:** Used to handle missing values in **categorical features (i.e, variables that take on a limited, discrete set of values).** 


## Arbitrary Number Imputer


- **Purpose:** Used for both **numerical** and **categorical** variables to fill in missing values with an **arbitrary value** that doesn't naturally occur in data. 


## Mean/Median Imputer


- **Purpose:** Used primarily for **numerical values** where missing values are replaced by the **mean** or **median** of the column.

In [ ]:
# Import the necessary imputation class
from feature_engine.imputation import CategoricalImputer

# Define the variables to impute
variables_to_impute = ["GarageFinish", "BsmtFinType1"]

# Instantiate the imputer with specific parameters
imputer = CategoricalImputer(imputation_method="missing", fill_value="Unf", variables=variables_to_impute)

# Apply the imputer to the DataFrame
df_imputed = imputer.fit_transform(df)

# Plot and compare the effect of imputation
plot_data_cleaning_effect(df_original=df, 
                          df_cleaned=df_imputed, 
                          variables_applied=variables_to_impute)

In [ ]:
# Import the necessary imputation class
from feature_engine.imputation import ArbitraryNumberImputer

# Define the variables to impute
variables_to_impute = ["2ndFlrSF", "EnclosedPorch", "MasVnrArea", "WoodDeckSF"]

# Instantiate the imputer with specific parameters
imputer = ArbitraryNumberImputer(arbitrary_number=0, variables=variables_to_impute)

# Apply the imputer to the DataFrame
df_imputed = imputer.fit_transform(df)

# Plot and compare the effect of imputation
plot_data_cleaning_effect(df_original=df, 
                          df_cleaned=df_imputed, 
                          variables_applied=variables_to_impute)

In [ ]:
# Import the necessary imputation class
from feature_engine.imputation import MeanMedianImputer

# Define the variables to impute
variables_to_impute = ["LotFrontage", "BedroomAbvGr", "GarageYrBlt"]

# Instantiate the imputer with specific parameters
imputer = MeanMedianImputer(imputation_method="median", variables=variables_to_impute)

# Apply the imputer to the DataFrame
df_imputed = imputer.fit_transform(df)

# Plot and compare the effect of imputation
plot_data_cleaning_effect(df_original=df, 
                          df_cleaned=df_imputed, 
                          variables_applied=variables_to_impute)

In [ ]:
# Shuffle the DataFrame
shuffled_df = df.sample(frac=1, random_state=0).reset_index(drop=True)

# Calculate the index for splitting
split_index = int(0.8 * len(shuffled_df))

# Create the training and testing sets
TrainSet = shuffled_df.iloc[:split_index]
TestSet = shuffled_df.iloc[split_index:]

# Get the SalePrice column
y_train = TrainSet["SalePrice"]
y_test = TestSet["SalePrice"]

print(f"TrainSet shape: {TrainSet.shape} \nTestSet shape: {TestSet.shape}")

In [23]:
# Define variables to impute
variables_to_impute =["GarageFinish", "BsmtFinType1"]

variables_method = variables_to_impute

# Create a CategoricalImputer instance with specified parameters
imputer = CategoricalImputer(
    imputation_method = "missing",
    fill_value = "Unf",
    variables=variables_method
)

# Fit the imputer on training set
imputer.fit(TrainSet)

# Apply the transformation to both the training and test sets
TrainSet = imputer.transform(TrainSet)
TestSet = imputer.transform(TestSet)

In [25]:
# Define variables to impute
variables_to_impute =["2ndFlrSF", "EnclosedPorch", "MasVnrArea", "WoodDeckSF"]

variables_method = variables_to_impute

# Create a ArbitraryNumberImputer instance with specified parameters
imputer = ArbitraryNumberImputer(
    arbitrary_number=0,
    variables=variables_method
)

# Fit the imputer on training set
imputer.fit(TrainSet)

# Apply the transformation to both the training and test sets
TrainSet = imputer.transform(TrainSet)
TestSet = imputer.transform(TestSet)

In [27]:
# Define variables to impute
variables_to_impute =["LotFrontage", "BedroomAbvGr", "GarageYrBlt"]

variables_method = variables_to_impute

# Create a MeanMedianImputer instance with specified parameters
imputer = MeanMedianImputer(
    imputation_method="median",
    variables=variables_method
)

# Fit the imputer on training set
imputer.fit(TrainSet)

# Apply the transformation to both the training and test sets
TrainSet = imputer.transform(TrainSet)
TestSet = imputer.transform(TestSet)

In [28]:
import os
try:
  os.makedirs(name='outputs/datasets/cleaned') 
except Exception as e:
  print(e)

In [29]:
TrainSet.to_csv("outputs/datasets/cleaned/TrainSetCleaned.csv", index=False)

In [30]:
TestSet.to_csv("outputs/datasets/cleaned/TestSetCleaned.csv", index=False)